In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import geopandas as gp
from datetime import date, datetime

In [ ]:
import matplotlib.pyplot as plt
from plot_tools import *
import cartopy.crs as ccrs
import seaborn as sns

## read data

In [ ]:
t0, t1 = datetime(1980,1,1), datetime(2014,12,30)

# criteria
q = 95
min_dist = 45
window_size = 7
qmin=0
rp = 20

model = 'mean' #anu
drivers = ['runoff', 'surge', 'cmpnd']

In [ ]:
ddir = r'/scratch/compound'
fig_dir = r'/scratch/compound/figs'

fn_peaks = join(ddir, f'peaks_q{q}d{min_dist}_wdw{window_size}.nc')
ds_peaks = xr.open_dataset(fn_peaks, chunks={'ensemble':-1, 'index':250, 'time': -1})
ds_peaks.ensemble.data = ds_peaks.ensemble.data.astype(str)

fn_peaks_stats = join(ddir, f'peaks_q{q}d{min_dist}_wdw{window_size}_stats.nc')
ds_peaks_stats = xr.open_dataset(fn_peaks_stats, chunks={'ensemble':-1, 'index':250})
ds_peaks_stats.ensemble.data = ds_peaks_stats.ensemble.data.astype(str)

fn_cmf = join(ddir, 'gcfr.zarr')
ds_cmf = xr.open_zarr(fn_cmf).sel(time=slice(t0,t1))
ds_cmf.ensemble.data = ds_cmf.ensemble.data.astype(str)

### reduce ensemble dim

In [ ]:
drop_var = [v for v in ds_peaks_stats.data_vars if ds_peaks_stats[v].ndim >= 3] + ['rp', 'scen']
if model == 'mean':
    ds_peaks_stats_mod = ds_peaks_stats.sel(rp=rp).drop(drop_var).mean('ensemble')
    peaks_all_n = ds_peaks_stats[f'peaks100_n'].sum('ensemble')
    for driver in drivers:
        peaks_driver_n = ds_peaks_stats[f'peaks_{driver}_n'].sum('ensemble')
        # ensemble mean
        ds_peaks_stats_mod[f'peaks_{driver}_perc'] = peaks_driver_n / peaks_all_n * 100
        ds_peaks_stats_mod[f'peaks_{driver}_n'] = peaks_driver_n / ds_peaks_stats['ensemble'].size
        # circular data
#         circ = xs.doy_to_circ(ds_peaks_stats[f'cfi_{driver}_doy_mean'], dim=None)
#         theta = xs.circ_stats.circ_mean(circ, dim='ensemble')
#         ds_peaks_stats_mod[f'cfi_{driver}_doy_mean'] = xs.circ_to_doy(theta, None)
#         ds_peaks_stats_mod[f'cfi_{driver}_doy_uniform_p'] = ds_peaks_stats[f'cfi_{driver}_doy_uniform_p'].max('ensemble')
        
else:
    ds_peaks_stats_mod = ds_peaks_stats.drop(drop_var).sel(ensemble=model)

df = ds_peaks_stats_mod.to_dataframe().rename(columns={'rivmth_lon':'lon', 'rivmth_lat':'lat'})
df.to_csv(join(ddir, f'cfi_q{q}d{min_dist}_wdw{window_size}_{model}_rp{rp}_stats.csv'), float_format='%.4f')

df = df[df['Qmean'] >= qmin]
df['Qmean_log10'] = np.log10(df['Qmean'])
df['markersize'] = np.maximum((df['Qmean_log10'] / df['Qmean_log10'].max()),0) * 40 + 2
gdf = pandas2geopandas(df, crs={'init': 'epsg:4326'})
N = len(df)
print(N)

## plot settings

In [ ]:
rc = {'savefig.bbox': 'tight',  'savefig.format': 'png', 'savefig.dpi':300}
context = 'paper'
# sns.set(context=context, style='whitegrid', font_scale=0.75 if context == 'talk' else 1., rc=rc)
sns.set(context=context, style='whitegrid', font_scale=1.2 if context == 'paper' else 1., rc=rc)

crs = ccrs.Robinson()
crs_sub = ccrs.PlateCarree()

cmap_div = sns.diverging_palette(220, 10, s=75, l=40, sep=1, as_cmap=True)

bmap_kwargs = dict(
    features=['land'],
    feat_colors = [cfeature.COLORS['land_alt1']],
)

plot_kwargs=dict(edgecolor=(0.5, 0.5, 0.5, 0.8), linewidth=0.5, legend=False, zorder=2)
box_kwargs=dict(whis=[5,95], showfliers=False, boxprops=dict(linewidth=1.5), medianprops=dict(linewidth=1.5))

regions = {
    'A': (-100.0, 22.0, -50.0, 53.0), # 30 x 18
    'B': (-12.0, 30.0, 38.0, 60.0), # 50 x 30
    'C': (100.0, 20.0, 150.0, 50.0),  # 50 x 30
}

In [ ]:
def interpolate(colormap, x):
    x = max(0.0, min(1.0, x))
    a = int(x*255.0)
    b = min(255, a + 1)
    f = x*255.0 - a
    return [colormap[a][0] + (colormap[b][0] - colormap[a][0]) * f,
          colormap[a][1] + (colormap[b][1] - colormap[a][1]) * f,
          colormap[a][2] + (colormap[b][2] - colormap[a][2]) * f]

cticks_norm = np.array([0.1, 0.2, 0.5, 0.6, 0.7, 0.8, 0.9, 1.])
cmap_data =[interpolate(google_turbo_data, x) for x in cticks_norm]
cmap2 = ListedColormap(cmap_data)
len(cmap_data)

#### map 1: peak percentage

In [ ]:
gdf['peaks_cmpnd_perc'].describe(percentiles=[0.1,0.25,0.75,0.9])

In [ ]:
plt.close('all')
fig = plt.figure(figsize=(17, 10))
grid = plt.GridSpec(2, 6, hspace=0.1, wspace=0.4)

column = 'peaks_cmpnd_perc'
vmin, vmax= 0, 100 
cticks=np.linspace(vmin, vmax, 11)
gdf1 = gdf.copy().sort_values(column, ascending=True)


# main map
axg = fig.add_subplot(grid[:-1, :-1], projection=crs_sub)
add_regions(axg, regions, dx=-6, dy=-6)
basemap(axg, bbox=(-180, -60, 180, 90), gridlines=True, outline=True, **bmap_kwargs)
plot_kwargs.update(markersize=gdf1['markersize'].values)
cax = plot_choropleth(
    fig, axg, gdf1, column=column, 
    cmap=cmap_turbo, vmin=vmin, vmax=vmax, cticks=cticks, discrete=True,
    plot_kwargs=plot_kwargs,
    cbar_kwargs=dict(label='percentage of water level peaks', location='right', extend='both'),
    cbar_pos = dict(pad=0.1, fraction=0.1)
)

# submaps 
gdf1 = gdf1.sort_values('Qmean_log10', ascending=True)
axs = dict()
plot_kwargs.update(markersize=gdf1['markersize'].values*1.5)
for i, (name, bbox) in enumerate(regions.items()):
    ax = fig.add_subplot(grid[-1, i*2:(i+1)*2], projection=crs_sub)
    basemap(ax, bbox=bbox, gridlines=True, gridspacing=10, outline=True, **bmap_kwargs)
    plot_choropleth(
        fig, ax, gdf1, column=column, 
        cmap=cmap_turbo, vmin=vmin, vmax=vmax, cticks=cticks, discrete=True,
        plot_kwargs=plot_kwargs, cbar_kwargs=False,
    )
    xmin, ymax = bbox[0], bbox[3]
    ax.text(xmin, ymax+1, name, transform=crs_sub)
    axs[name] = ax
    
# boxplot ensemble
box_kwargs.update(widths=0.5)
color_props = dict(boxes="lightgrey", whiskers="k", medians="k", caps="k")
ax = fig.add_subplot(grid[:-1, -1:])
df_box = ds_peaks_stats[column].sel(index=gdf.index.values).to_series().unstack(-1).T * 100.
df_box.plot.box(ax=ax, color=color_props, patch_artist=True, **box_kwargs)
ax.set_ylim([cticks[1], cticks[-2]])
ax.yaxis.set_ticks(cticks[1:-2])
ax.yaxis.set_ticklabels([])
ax.grid(axis='x')
ax.set_xlabel('model')
cpos = cax.get_position()
bpos = ax.get_position()
cax.set_position([cpos.x0, cpos.y0, cpos.width, cpos.height])
# ax.set_position([cpos.x0+0.03, cpos.y0, bpos.width*1.5, cpos.height])
ax.set_position([cpos.x0+0.03, cpos.y0+cpos.height*0.05, bpos.width*1.5, cpos.height/1.10])
cax.yaxis.set_ticks_position('left')
cax.yaxis.set_label_position('left')

plt.savefig(join(fig_dir, f'peaks_q{q}d{min_dist}_wdw{window_size}_perc.png'))

#### map 2: compound ratio

In [ ]:
idx_sign = np.logical_and(gdf['compound_ratio_sign']==1, gdf['compound_ratio_dir']==1)

idx_sign.sum()/idx_sign.size, (gdf[idx_sign]['compound_ratio_mean']*100).describe(percentiles=[0.1,0.25,0.75,0.9])

In [ ]:
plt.close('all')
fig = plt.figure(figsize=(17, 10))
grid = plt.GridSpec(2, 6, hspace=0.1, wspace=0.4)

column = 'compound_ratio_mean'
vmin, vmax= -0.10, .30
cticks=np.linspace(vmin, vmax, 9)
idx_sign = np.logical_and(gdf['compound_ratio_sign']==1, gdf['compound_ratio_dir']==1)
idx_runoff = gdf[idx_sign]['main_driver']==-1
idx_surge = gdf[idx_sign]['main_driver']==1
gdf0 = gdf[idx_sign==False].copy()
gdf1 = gdf[idx_sign].copy().sort_values(column, ascending=True)

# main map
axg = fig.add_subplot(grid[:-1, :-1], projection=crs_sub)
add_regions(axg, regions, dx=-6, dy=-6)
basemap(axg, bbox=(-180, -60, 180, 90), gridlines=True, outline=True, **bmap_kwargs)
gdf0.plot(ax=axg, marker='x', color=(0, 0, 0, 0.8), markersize=5, linewidth=0.2, legend=False)

plot_kwargs.update(markersize=gdf1[idx_surge]['markersize'].values, marker='^')
plot_choropleth(
    fig, axg, gdf1[idx_surge], column=column, 
    cmap=cmap2, vmin=vmin, vmax=vmax, cticks=cticks, discrete=True,
    plot_kwargs=plot_kwargs, cbar_kwargs=False
)
plot_kwargs.update(markersize=gdf1[idx_runoff]['markersize'].values, marker='o')
cax = plot_choropleth(
    fig, axg, gdf1[idx_runoff], column=column, 
    cmap=cmap2, vmin=vmin, vmax=vmax, cticks=cticks, discrete=True,
    plot_kwargs=plot_kwargs,
    cbar_kwargs=dict(label=f'factor change return level (rp {rp} year) [-]', location='right', extend='both'),
    cbar_pos = dict(pad=0.1, fraction=0.1)
)

# submaps 
axs = dict()
gdf1 = gdf1.sort_values('Qmean_log10', ascending=True)
for i, (name, bbox) in enumerate(regions.items()):
    ax = fig.add_subplot(grid[-1, i*2:(i+1)*2], projection=crs_sub)
    basemap(ax, bbox=bbox, gridlines=True, gridspacing=10, outline=True, **bmap_kwargs)
    gdf0.plot(ax=ax, marker='x', color=(0, 0, 0, 0.8), markersize=15, linewidth=0.2, legend=False)
    plot_kwargs.update(markersize=gdf1[idx_surge]['markersize'].values*2, marker='^')
    plot_choropleth(
        fig, ax, gdf1[idx_surge], column=column, 
        cmap=cmap2, vmin=vmin, vmax=vmax, cticks=cticks, discrete=True,
        plot_kwargs=plot_kwargs, cbar_kwargs=False
    )
    plot_kwargs.update(markersize=gdf1[idx_runoff]['markersize'].values*2, marker='o')
    plot_choropleth(
        fig, ax, gdf1[idx_runoff], column=column, 
        cmap=cmap2, vmin=vmin, vmax=vmax, cticks=cticks, discrete=True,
        plot_kwargs=plot_kwargs, cbar_kwargs=False
    )
    xmin, ymax = bbox[0], bbox[3]
    ax.text(xmin, ymax+1, name, transform=crs_sub)
    axs[name] = ax
    
# boxplot ensemble
ax2 = fig.add_subplot(grid[:-1, -1:])
# surge
box_kwargs.update(widths=0.3)
color_props = dict(boxes="lightgreen", whiskers="k", medians="k", caps="k")
df_box = ds_peaks_stats[f'compound_ratio'].sel(rp=rp, index=gdf1[idx_surge].index).to_series().unstack(-1).T
df_box.plot.box(ax=ax2, positions=np.arange(5)-0.2, color=color_props, patch_artist=True, **box_kwargs)
# runoff
color_props = dict(boxes="lightblue", whiskers="k", medians="k", caps="k")
df_box = ds_peaks_stats[f'compound_ratio'].sel(rp=rp, index=gdf1[idx_runoff].index).to_series().unstack(-1).T
df_box.plot.box(ax=ax2, positions=np.arange(5)+0.2, color=color_props, patch_artist=True, **box_kwargs)
Qpatch = mpl.patches.Patch(color='lightblue', label='discharge')
Hpatch = mpl.patches.Patch(color='lightgreen', label='surge')
ax2.legend(handles=[Hpatch, Qpatch], ncol=2)

ax2.set_ylim([cticks[1], cticks[-2]])
ax2.yaxis.set_ticks(cticks[1:-2])
ax2.yaxis.set_ticklabels([])
ax2.grid(axis='x')
ax2.set_xlabel('model')
cpos = cax.get_position()
bpos = ax2.get_position()
cax.set_position([cpos.x0, cpos.y0, cpos.width, cpos.height])
ax2.set_position([cpos.x0+0.03, cpos.y0+cpos.height*0.05, bpos.width*1.5, cpos.height/1.1])
cax.yaxis.set_ticks_position('left')
cax.yaxis.set_label_position('left')

plt.savefig(join(fig_dir, f'peaks_q{q}d{min_dist}_wdw{window_size}_rp{rp:02d}_ratio.png'))

#### 3 plot per location

In [ ]:
import xhydrostats as xs 
import scipy
import pdb

R = np.array([0.5, 1, 2, 5, 10, 15, 20, 25, 30, 34])
R = np.arange(1,35)
nyears = 35

def _bootstrap_indexes(data, n_samples=1000):
    return np.random.randint(data.shape[0], size=(n_samples, data.shape[0]))

def _peaks2ev_single(peaks, peaks_n, peaks_lambda, R):
    peaks_rank = scipy.stats.rankdata(-1. * peaks, method='ordinal')
    peaks_prop = peaks_rank/float((peaks_n/peaks_lambda)-1) # weibull plotting position
    return scipy.interpolate.interp1d(peaks_prop, peaks, bounds_error=False)(1./R)

def _peaks2ev(peaks, nyears=nyears, R=R, ci=None, n_samples=10000):
    peaks = peaks[np.isfinite(peaks)]
    peaks_n = peaks.size
    peaks_lambda = 1.
    if nyears is not None:
        peaks_lambda = peaks_n / nyears 
    if ci is not None:
        peaks = peaks[_bootstrap_indexes(peaks, n_samples=n_samples)]
        peaks_ev = np.percentile(
            np.apply_along_axis(_peaks2ev_single, -1, peaks, peaks_n, peaks_lambda, R),
            ci, axis=0
        )
    else:
        peaks_ev = _peaks2ev_single(peaks, peaks_n, peaks_lambda, R)
    return peaks_ev

def _return_level(*args, **kwargs):
    return np.apply_along_axis(_peaks2ev, -1, *args, **kwargs)



In [ ]:
import xhydrostats as xs 
import scipy
import pdb

def _peaks2ev_single(peaks, peaks_n, peaks_lambda, R):
    peaks_rank = scipy.stats.rankdata(-1. * peaks, method='ordinal')
    peaks_prop = peaks_rank/float((peaks_n/peaks_lambda)-1) # weibull plotting position
    return scipy.interpolate.interp1d(peaks_prop, peaks, bounds_error=False)(1./R)

def _peaks2ev(peaks, nyears, R, axis=None):
    peaks = peaks[np.isfinite(peaks)]
    peaks_n = peaks.size
    peaks_lambda = 1.
    if nyears is not None:
        peaks_lambda = peaks_n / nyears 
#     print(peaks.size, peaks_n, peaks_lambda)
    return _peaks2ev_single(peaks, peaks_n, peaks_lambda, R)

def _return_level(arr, axis=-1, *args, **kwargs):
    return np.apply_along_axis(_peaks2ev, arr=arr, axis=axis, *args, **kwargs)



In [ ]:
def _ci(data, statfunction=np.nanmean, alpha=0.05, n_samples=10000, method='bca', statkwargs={}):
    assert data.ndim==1, "only tested for 1D arrays"
    data = data[np.isfinite(data)] # remove nans
    n = data.size
    assert n>0, "empty array"
    alphas = np.array([alpha/2, 1-alpha/2])
    stat = statfunction(data[np.random.randint(n, size=(n, n_samples))], axis=0, **statkwargs)
    print(stat.shape)
    assert stat.shape[-1] == n_samples
    stat = np.apply_along_axis(np.sort, arr=stat, axis=0)
    print(stat.shape)

    if method == 'simple':
        avals = alphas

    elif method == 'bca':
        if stat.ndim > 1:
            raise NotImplementedError('bca method only implemented for statfunctions which return scalar values')
        ostat = statfunction(data, axis=0, **statkwargs)
        # The bias correction value.
        z0 = norm.ppf(( 1.0*np.sum(stat < ostat, axis=0)) / n_samples )
        # Statistics of the jackknife distribution
        jstat = statfunction(data[_jackknife_sample(np.arange(n, dtype=np.int16))], axis=0, **statkwargs)
        jmean = jstat.mean(axis=0)
        # Acceleration value
        av = np.sum((jmean - jstat)**3, axis=0) / (6.0 * np.sum((jmean - jstat)**2, axis=0)**1.5)
        zs = z0 + norm.ppf(alphas).reshape(alphas.shape+(1,)*z0.ndim)
        # Bias corrected alphas
        avals = norm.cdf(z0 + zs/(1-av*zs))

    #get confidence interval based on sorted bootstrapped statistic
    nvals = np.nan_to_num(np.round((n_samples-1)*avals)).astype('int')

    return stat[..., nvals]

In [ ]:

def confidence_interval(da, statfunction=np.nanmean, alpha=0.05, 
    n_samples=10000, method='bca', dim='time', statkwargs={}):
    # confidence interval parameters
    ci_kwargs = dict(
        statfunction=statfunction, 
        alpha=alpha, 
        n_samples=n_samples, 
        method=method,
        statkwargs=statkwargs
        )
    # apply ufunc parameters
    output_sizes = {d:da[d].size for d in da.reset_coords(drop=True).coords if d != dim}
    if 'R' in statkwargs:
        output_sizes.update(rp= np.atleast_1d(statkwargs.get('R')).size)
    kwargs = dict(               
        input_core_dims=[[dim]], 
        output_core_dims=[['ci_bounds', 'rp']],
        dask='allowed', 
        output_dtypes=[np.float],       
        # on output, <dim> is reduced to length 2 with upper and lower ci bounds
        output_sizes={'ci_bounds': 2, 'rp': 2},
        # vectorize to apply over 1 dim
        vectorize=True
    )
    print(kwargs)
    # apply ci_nd over dim
    ci = xr.apply_ufunc(_ci, da, kwargs=ci_kwargs, **kwargs)
    ci['ci_bounds'] = xr.Variable('ci_bounds', np.array([alpha/2, 1-alpha/2]))
    return ci

In [ ]:
ds_test = ds_cmf['water_level'].isel(index=slice(2), ensemble=slice(2)).load()

In [ ]:
confidence_interval(ds_test.chunk({'time':-1}), statfunction=_return_level, n_samples=100, method='simple', 
                    statkwargs=dict(R=np.array([2., 10.]), nyears=35))

In [ ]:
a = np.array([[4, 2], 
              [3, 5],
              [2, 1]])
np.apply_along_axis(np.sort, arr=a, axis=0)

In [ ]:
nyears= 35
loc = 1617
model='cnrs'
rm = {'cmpnd': 'compound', 'runoff': 'discharge'}
rm_vars = {'surge_wdw_max': 'y', 'runoff_wdw_max': 'x', 'peaks_all': 'i'}
ds_loc = xr.merge([ds_peaks.rename(rm_vars)[['x', 'y', 'i']], ds_cmf['water_level']]).sel(index=loc, ensemble=model)
ds_loc.scen.data = [rm.get(n, n) for n in ds_loc.scen.data]
ds_loc = ds_loc.where(ds_loc['i'])
h = ds_loc['water_level'].isel(scen=0).values #.dropna('time', how = 'all').values
h

In [ ]:
_return_level(h, nyears=35, R=10)

In [ ]:
xs.bootstrap._ci(h, statfunction=_return_level, n_samples=10000, statkwargs=dict(R=10, nyears=35))

In [ ]:
import scipy
def _peaks2rp_single(peaks, nyears):
    valid = np.isfinite(peaks)
    rp = np.ones(peaks.size)*np.nan
    peaks = peaks[valid]
    peaks_n = valid.size
    peaks_lambda = 1.
    if nyears is not None:
        peaks_lambda = peaks_n / nyears 
    peaks_rank = scipy.stats.rankdata(-1. * peaks, method='ordinal')
    peaks_prop = peaks_rank/float((peaks_n/peaks_lambda)-1) # weibull plotting position
    rp[valid] = 1/peaks_prop
    return rp

In [ ]:

xs.confidence_interval(ds_loc['water_level'], statfunction=_peaks2rp)

In [ ]:
# R = np.array([1.1, 1.2, 1.5, 2, 3, 4, 5, 10, 15, 20, 25, 30, 34])
# nyears = 35
# def _peaks2ev_single(peaks, peaks_n, peaks_lambda, R):
#     peaks_rank = scipy.stats.rankdata(-1. * peaks, method='ordinal')
#     peaks_prop = peaks_rank/float((peaks_n/peaks_lambda)-1) # weibull plotting position
#     return scipy.interpolate.interp1d(peaks_prop, peaks, bounds_error=False)(1./R)

# def _peaks2ev(peaks, nyears, R, ci=[5, 50, 95], n_samples=10000):
#     peaks = peaks[np.isfinite(peaks)]
#     peaks_n = peaks.size
#     peaks_lambda = 1.
#     if nyears is not None:
#         peaks_lambda = peaks_n / nyears 
#     if ci is not None:
#         peaks = peaks[_bootstrap_indexes(peaks, n_samples=n_samples)]
#         peaks_ev = np.percentile(
#             np.apply_along_axis(_peaks2ev_single, -1, peaks, peaks_n, peaks_lambda, R),
#             ci, axis=0
#         )
#     else:
#         peaks_ev = _peaks2ev_single(peaks, peaks_n, peaks_lambda, R)
#     return peaks_ev

In [ ]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
# [1617, 1230, 483, 1646]
nyears= 35
loc = 1617
model='cnrs'
rm = {'cmpnd': 'compound', 'runoff': 'discharge'}
scen_cmap={'compound': 'darkorange', 'discharge': 'green', 'surge': 'blue'}
scen_mmap={'compound': '^', 'discharge': 'o', 'surge': 'x'}
ls={'compound': '-', 'discharge': '--', 'surge': '-.'}
scenarios = ['surge', 'discharge', 'compound']


In [ ]:
from matplotlib.colors import BoundaryNorm, ListedColormap

rm_vars = {'surge_wdw_max': 'y', 'runoff_wdw_max': 'x', 'peaks_all': 'i'}
ds_loc = xr.merge([ds_peaks.rename(rm_vars)[['x', 'y', 'i']], ds_cmf['water_level']]).sel(index=loc, ensemble=model)
ds_loc.scen.data = [rm.get(n, n) for n in ds_loc.scen.data]
ds_loc = ds_loc.where(ds_loc['i'])
for scen in ds_loc.scen.data:
    ds_loc[f'rp_{scen}'] = xr.Variable('time', _peaks2rp_single(ds_loc['water_level'].sel(scen=scen).values, nyears))
    
# get compound water levels and drivers
scen = 'compound'
ds_loc_scen = ds_loc[['x', 'y', 'i', f'rp_{scen}']].sel(scen=scen).rename({f'rp_{scen}': 'rp'})
del_vars = [v for v in ds_loc_scen.coords if v != 'time']
ds_loc_scen = ds_loc_scen.where(ds_loc_scen['i']==1, drop=True).drop(del_vars)
df = ds_loc_scen.to_dataframe()


rps = [0.2, 1, 2, 5, 10, 20, 30]
cmap = ListedColormap(google_turbo_data[50:])
cmap.set_over('black')
norm = BoundaryNorm(rps, cmap.N)

fig, (ax, ax2) = plt.subplots(1, 2, figsize=(17,5))

df.plot.scatter(ax=ax, y='y', x='x', c='rp', s=df['rp'].values+25, cmap=cmap, norm=norm, colorbar=False)
cax, _ = mpl.colorbar.make_axes_gridspec(ax, orientation='vertical', fraction=0.2)
cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap, norm=norm, extend='max')
xlim = ax.get_xlim()
ylim = ax.get_ylim()
ax.hlines(0, *xlim, linewidth=0.5, linestyle='--')
ax.vlines(0, *ylim, linewidth=0.5, linestyle='--')
ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_xlabel('discharge anomaly [m$^3$/s]')
ax.set_ylabel('surge [m]')
cax.set_ylabel('return period [years]')
# ax.set_title(f'compound POT (threshold = {q}th percentile)')

# rps = [2, 5, 10, 20, 35]
for scen in ds_loc.scen.data:
    ds_loc_scen = ds_loc[['water_level', f'rp_{scen}', 'i']].sel(scen=scen).rename({f'rp_{scen}': 'rp'})
    ds_loc_scen = ds_loc_scen.where(ds_loc_scen['i']==1, drop=True).drop(del_vars)
    x = ds_loc_scen['rp'].values
    y = ds_loc_scen['water_level'].values
#     ax2.plot(R, _return_level(y), c=scen_cmap[scen])
    ci = _return_level(y, ci=[5, 95])
    kwargs = dict(linewidth=1, color=scen_cmap[scen], linestyle=':', zorder=1)
    for i in range(ci.shape[0]):
        ax2.plot(R, ci[i,:], **kwargs)
    ax2.scatter(x=x, y=y, label=scen, c=scen_cmap[scen], marker=scen_mmap[scen], zorder=2)
ax2.legend()
ax2.set_xscale('log')
ax2.set_xlim([0.25, 40])
ax2.set_xticks(rps)
ax2.set_xticklabels(rps)
ax2.set_xlabel('return period [years]')
ax2.set_ylabel('water level [m +EGM96]')

# plt.savefig(join(fig_dir, f'peaks_q{q}d{min_dist}_wdw{window_size}_loc{loc}_{model}.png'))

In [ ]:
plt.close('all')

tslice = slice('12-01-2002', '06-01-2003')
ds_loc = ds_cmf.sel(index=loc, ensemble=model, time=tslice)
ds_loc.scen.data = [rm.get(n, n) for n in ds_loc.scen.data]

# plot timeseries 
for d in scenarios:
    i = 0
    fig, axs = plt.subplots(1, 3, figsize=(17, 5), sharex=True)
    ds_loc['sea_water_level'].plot.line(x='time', ax=axs[0], label='tide + seas. SL + surge', 
        c='blue' if d!='discharge' else 'grey', zorder = 2 if d!='discharge' else 1)
    ds_loc['sea_water_level_climatology'].plot.line(x='time', ax=axs[0], label='tide + seas. SL', 
        c='blue' if d=='discharge' else 'grey', zorder = 2 if d=='discharge' else 1, linestyle='--')
    axs[0].set_ylabel('water level [m+EGM96]')
    axs[0].legend(ncol=2, loc='lower center')
    
    for dd in scenarios:
        ds_loc['water_level'].sel(scen=dd).plot.line(x='time', ax=axs[1], label=dd, add_legend=False,
            linestyle=ls[dd], c=scen_cmap[dd] if dd==d else 'lightgrey', zorder = 2 if dd==d else 1)
    axs[1].set_ylabel('water level [m+EGM96]')
    axs[1].legend(ncol=3, loc='lower center')

    ds_loc['river_discharge'].plot.line(x='time', ax=axs[2], label='discharge',
            c='green' if d != 'surge' else 'grey', zorder = 2 if d != 'surge' else 1)
    ds_loc['river_discharge_climatology'].plot.line(x='time', ax=axs[2], linestyle='--', label='discharge climatology',
            c='green' if d == 'surge' else 'grey', zorder = 2 if d == 'surge' else 1, )
    axs[2].set_ylabel('discharge [m$^3$/s]')
    axs[2].legend(ncol=2, loc='lower center')

    axs[0].set_title('downstream')
    axs[1].set_title('river mouth')
    axs[2].set_title('upstream')
    
    fig.suptitle(d, size=14, weight='bold')
    
    plt.savefig(join(fig_dir, f'ts_{model}_loc{loc}_{d}.png'))    
